In [1]:
from gradient_descent import gradient_descent, minibatch_class
import utils
from utils import Poisson_reg
from utils import sample_PLN

import matplotlib.pyplot as plt
from pandas import read_csv
import math
from timeit import default_timer as timer
from tqdm import tqdm 
import time 
import sys 

from __future__ import print_function
import psutil
import multiprocessing
import concurrent.futures
import threading

import numpy as np
import torch
from torch import nn
import scipy.linalg as SLA 
from scipy.linalg import toeplitz 
torch.set_default_dtype(torch.float64)

In [100]:
class PLN_full():
    def __init__(self, C_init, beta_init, M_init, S_init, requires_tridiag = True): 
        '''
            Initialization : 
            'Y' : the data, size (n,p). n is the number of samples we have and p the number of species. 
                  the entries should be int but dtype = float. 
            'O': offset : additional offset. (not very important for comprehension). size (n,p)
            'covariates' : covariates, size (n,d)
            'C_init' : initialization for C. I plan to do a more advanced initialization. 
            'beta_init ' : Initialization for beta. I plan to do a more advanced initialization. 
            'M_init' : initialization for the variational parameter M
            'S_init ': initialization for the variational parameter S
        '''
        # model parameters
        
        # we start with the initialization of C, that is the most subtle. If you want tridiagonals matrices 
        # at every iteration then the parameters C of the optimizer will be a vector ( we will change this vector 
        # into a matrix at each iteration. It is more convenient this way. 
        #Else, it will be a matrix

        
        if requires_tridiag : 
            self.C = torch.clone(mat2vec_lower_tridiag(C_init))
            self.requires_tridiag = True 
        else : 
            self.C = torch.clone(C_init) 
            self.requires_tridiag = False 
            
        self.C.requires_grad_(True)
        
        self.beta = torch.clone(beta_init)
        self.beta.requires_grad_(True)
        
        self.Sigma = torch.clone(torch.mm(C_init,C_init.T))
        self.Sigma.requires_grad_(False) # we don't need to have a gradient for Sigma since we will 
                                         # always update it with a closed form (since we have constraints
                                         # of positivity and symmetry)
        #variational parameters
        self.M = torch.clone(M_init)
        self.M.requires_grad_(True)
        self.S = torch.clone(S_init) 
        self.S.requires_grad_(True)
        
        
        
        self.params = {'S' : self.S,'M' : self.M,'C' : self.C, 'beta' : self.beta}
        
        self.optimizer = torch.optim.Adam(self.params.values(), lr = 0.002)
        
        self.params['Sigma'] = self.Sigma
        
        # some list to store some stats
        self.MSE_Sigma_list = list()
        self.MSE_beta_list = list()
        self.ELBO_list = list()
        self.running_times = list()

        
    def extract_data(self,data): 
        '''
        function to extract the data. This function is just here to have a code more compact. 
        
        args : 
              'data': list with 3 elements : Y, O and covariates in this order. 
        '''
        #known variables
        self.Y = data[0];self.O = data[1];self.covariates = data[2]
        self.n, self.p = self.Y.shape
        
    def compute_ELBO(self): 
        return ELBO(self.Y,self.O , self.covariates,self.M ,self.S ,self.C ,self.beta)
    
    def compute_ELBO_tridiag(self): 
        return ELBO_tridiag(self.Y,self.O , self.covariates,self.M ,self.S ,self.C ,self.beta)
    def compute_ELBO_Sigma(self): 
        #we update Sigma
        #self.Sigma = 1/self.n*(torch.sum(torch.stack([torch.outer(self.M[i,:],self.M[i,:]) + torch.diag(torch.multiply(self.S,self.S)[i,:]) 
         #                              for i in range(self.n)]), axis = 0))
        
        
        return ELBO_Sigma(self.Y,self.O , self.covariates,self.M ,self.S ,self.Sigma ,self.beta)
    
    def full_grad_ascent(self,data, lr = 0.1, tolerance = 0, N_epoch = 1000, verbose = True ): 
        self.extract_data(data)
        if self.requires_tridiag : 
            loss = self.compute_ELBO_tridiag
        else : 
            loss = self.compute_ELBO
        
        self.last_params, self.running_times, self.MSE_Sigma_list, self.MSE_beta_list, self.ELBO_list = torch_gradient_ascent(self.optimizer, 
                            loss,self.params, lr = lr, tolerance = tolerance, N_epoch = N_epoch, requires_closed_beta= True, 
                            requires_closed_Sigma= False,verbose = verbose)
    def full_grad_ascent_Sigma(self,data, lr = 0.1, tolerance = 0, N_epoch = 1000, verbose = True ): 
        self.extract_data(data)
        self.last_params, self.running_times, self.MSE_Sigma_list, self.MSE_beta_list, self.ELBO_list = torch_gradient_ascent(self.optimizer, 
                            self.compute_ELBO_Sigma,self.params, lr = lr, tolerance = tolerance, N_epoch = N_epoch, requires_closed_beta= True, 
                            requires_closed_Sigma= True, verbose = verbose)
    

In [132]:
model = PLN_full(C_init, beta_init, M_init, S_init, requires_tridiag = True)
%time model.full_grad_ascent(data, N_epoch = 3000,verbose=True, lr = 0.3)

ELBO :  tensor(3072.9773)
ELBO :  tensor(5799.7783)
ELBO :  tensor(7035.9082)
ELBO :  tensor(8206.9694)
ELBO :  tensor(9557.8230)
ELBO :  tensor(10867.8923)
ELBO :  tensor(11984.9958)
ELBO :  tensor(12879.9921)
ELBO :  tensor(13575.3760)
ELBO :  tensor(14278.5961)
ELBO :  tensor(14827.5485)
ELBO :  tensor(15202.3746)
ELBO :  tensor(15480.2919)
ELBO :  tensor(15682.6575)
ELBO :  tensor(15764.1106)
ELBO :  tensor(15744.9247)
ELBO :  tensor(15679.4663)
ELBO :  tensor(15583.3160)
ELBO :  tensor(15581.3426)
ELBO :  tensor(15647.9769)
ELBO :  tensor(15728.0926)
ELBO :  tensor(15799.9619)
ELBO :  tensor(15876.9153)
ELBO :  tensor(15977.9631)
ELBO :  tensor(16083.3289)
ELBO :  tensor(16167.8074)
ELBO :  tensor(16210.6954)
ELBO :  tensor(16242.1734)
ELBO :  tensor(16238.0724)
ELBO :  tensor(16221.6459)
ELBO :  tensor(16201.9298)
ELBO :  tensor(16184.6105)
ELBO :  tensor(16175.6664)
ELBO :  tensor(16178.1339)
ELBO :  tensor(16191.5600)
ELBO :  tensor(16211.6633)
ELBO :  tensor(16238.8548)
ELBO :

ELBO :  tensor(16514.8761)
iteration number:  300
-------UPDATE-------
 MSE with Sigma :  174.7447
 MSE with beta :  0.74734
ELBO :  16514.87608
---------------------------------lr : 0.3
---------------------------------log likelihood : 16514.876082444818
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(

ELBO :  tensor(16514.8748)
ELBO :  tensor(16514.8755)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8758)
ELBO :  tensor(16514.8753)
ELBO :  tensor(16514.8753)
ELBO :  tensor(16514.8758)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8759)
ELBO :  tensor(16514.8756)
ELBO :  tensor(16514.8756)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8759)
ELBO :  tensor(16514.8758)
ELBO :  tensor(16514.8758)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8759)
ELBO :  tensor(16514.8759)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8761)
ELBO :  tensor(16514.8761)
E

ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8759)
ELBO :  tensor(16514.8759)
ELBO :  tensor(16514.8759)
ELBO :  tensor(16514.8758)
ELBO :  tensor(16514.8757)
ELBO :  tensor(16514.8756)
ELBO :  tensor(16514.8753)
ELBO :  tensor(16514.8751)
ELBO :  tensor(16514.8748)
ELBO :  tensor(16514.8744)
ELBO :  tensor(16514.8744)
ELBO :  tensor(16514.8744)
ELBO :  tensor(16514.8748)
ELBO :  tensor(16514.8753)
ELBO :  tensor(16514.8758)
ELBO :  tensor(16514.8759)
ELBO :  tensor(16514.8758)
ELBO :  tensor(16514.8755)
ELBO :  tensor(16514.8753)
ELBO :  tensor(16514.8753)
ELBO :  tensor(16514.8754)
ELBO :  tensor(16514.8755)
ELBO :  tensor(16514.8756)
ELBO :  tensor(16514.8754)
ELBO :  tensor(16514.8752)
ELBO :  tensor(16514.8750)
ELBO :  tensor(16514.8750)
ELBO :  tensor(16514.8752)
ELBO :  tensor(16514.8756)
ELBO :  tensor(16514.8759)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8758)
ELBO :  tensor(16514.8757)
ELBO :  tensor(16514.8756)
ELBO :  tensor(16514.8757)
E

ELBO :  tensor(16514.8749)
ELBO :  tensor(16514.8750)
ELBO :  tensor(16514.8751)
ELBO :  tensor(16514.8753)
ELBO :  tensor(16514.8754)
ELBO :  tensor(16514.8756)
ELBO :  tensor(16514.8757)
ELBO :  tensor(16514.8758)
ELBO :  tensor(16514.8759)
ELBO :  tensor(16514.8759)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8759)
ELBO :  tensor(16514.8759)
ELBO :  tensor(16514.8758)
ELBO :  tensor(16514.8758)
ELBO :  tensor(16514.8758)
ELBO :  tensor(16514.8758)
ELBO :  tensor(16514.8758)
ELBO :  tensor(16514.8759)
ELBO :  tensor(16514.8759)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8760)
ELBO :  tensor(16514.8759)
ELBO :  tensor(16514.8758)
ELBO :  tensor(16514.8754)
ELBO :  tensor(16514.8744)
ELBO :  tensor(16514.8723)
ELBO :  tensor(16514.8681)
ELBO :  tensor(16514.8669)
ELBO :  tensor(16514.8700)
E

ELBO :  tensor(16512.0618)
ELBO :  tensor(16509.8107)
ELBO :  tensor(16511.8891)
ELBO :  tensor(16514.4624)
ELBO :  tensor(16514.0691)
ELBO :  tensor(16512.0819)
ELBO :  tensor(16509.3142)
ELBO :  tensor(16511.1767)
ELBO :  tensor(16513.9313)
ELBO :  tensor(16513.8582)
ELBO :  tensor(16512.2289)
ELBO :  tensor(16509.6756)
ELBO :  tensor(16511.6817)
ELBO :  tensor(16514.4766)
ELBO :  tensor(16514.3234)
ELBO :  tensor(16512.5624)
ELBO :  tensor(16509.9128)
ELBO :  tensor(16511.8770)
ELBO :  tensor(16514.6389)
ELBO :  tensor(16514.3232)
ELBO :  tensor(16512.3881)
ELBO :  tensor(16509.6271)
ELBO :  tensor(16511.6758)
ELBO :  tensor(16514.6537)
ELBO :  tensor(16514.4692)
ELBO :  tensor(16512.5409)
ELBO :  tensor(16509.6790)
ELBO :  tensor(16511.6832)
ELBO :  tensor(16514.6885)
ELBO :  tensor(16514.4636)
ELBO :  tensor(16512.4457)
ELBO :  tensor(16509.4747)
ELBO :  tensor(16511.4923)
ELBO :  tensor(16514.6062)
ELBO :  tensor(16514.4069)
ELBO :  tensor(16512.3297)
ELBO :  tensor(16509.2795)
E

ELBO :  tensor(16514.7011)
ELBO :  tensor(16514.6998)
ELBO :  tensor(16514.7628)
ELBO :  tensor(16514.8112)
ELBO :  tensor(16514.8203)
ELBO :  tensor(16514.8195)
ELBO :  tensor(16514.8163)
ELBO :  tensor(16514.8056)
ELBO :  tensor(16514.7912)
ELBO :  tensor(16514.7854)
ELBO :  tensor(16514.8069)
ELBO :  tensor(16514.8191)
ELBO :  tensor(16514.8165)
ELBO :  tensor(16514.8108)
ELBO :  tensor(16514.8150)
ELBO :  tensor(16514.8066)
ELBO :  tensor(16514.7998)
ELBO :  tensor(16514.7650)
ELBO :  tensor(16514.7678)
ELBO :  tensor(16514.7476)
ELBO :  tensor(16514.7592)
ELBO :  tensor(16514.7359)
ELBO :  tensor(16514.7478)
ELBO :  tensor(16514.7304)
ELBO :  tensor(16514.7455)
ELBO :  tensor(16514.7346)
ELBO :  tensor(16514.7324)
ELBO :  tensor(16514.7015)
ELBO :  tensor(16514.6605)
ELBO :  tensor(16514.5797)
ELBO :  tensor(16514.4948)
ELBO :  tensor(16514.4018)
ELBO :  tensor(16514.3684)
ELBO :  tensor(16514.3991)
ELBO :  tensor(16514.5344)
ELBO :  tensor(16514.7013)
ELBO :  tensor(16514.8174)
E

ELBO :  tensor(16462.1878)
ELBO :  tensor(16503.9676)
ELBO :  tensor(16504.1638)
ELBO :  tensor(16484.5881)
ELBO :  tensor(16469.8970)
ELBO :  tensor(16468.8653)
ELBO :  tensor(16480.1353)
ELBO :  tensor(16498.3911)
ELBO :  tensor(16512.9968)
ELBO :  tensor(16510.5809)
ELBO :  tensor(16494.8306)
ELBO :  tensor(16488.8962)
ELBO :  tensor(16500.8011)
ELBO :  tensor(16512.6952)
ELBO :  tensor(16513.5240)
ELBO :  tensor(16507.3210)
ELBO :  tensor(16502.2560)
ELBO :  tensor(16501.8795)
ELBO :  tensor(16505.1694)
ELBO :  tensor(16510.6558)
ELBO :  tensor(16513.8186)
ELBO :  tensor(16512.7608)
ELBO :  tensor(16509.4604)
ELBO :  tensor(16508.1027)
ELBO :  tensor(16510.4452)
ELBO :  tensor(16513.1053)
ELBO :  tensor(16513.8157)
ELBO :  tensor(16513.6117)
ELBO :  tensor(16512.3003)
ELBO :  tensor(16511.6008)
ELBO :  tensor(16511.9850)
ELBO :  tensor(16513.0085)
ELBO :  tensor(16513.9213)
ELBO :  tensor(16514.1385)
ELBO :  tensor(16513.8630)
ELBO :  tensor(16513.5440)
ELBO :  tensor(16513.5026)
E

ELBO :  tensor(16514.4757)
ELBO :  tensor(16514.4493)
ELBO :  tensor(16514.2904)
ELBO :  tensor(16514.2466)
ELBO :  tensor(16514.0130)
ELBO :  tensor(16514.0227)
ELBO :  tensor(16513.7459)
ELBO :  tensor(16513.6336)
ELBO :  tensor(16513.4598)
ELBO :  tensor(16513.0929)
ELBO :  tensor(16512.9564)
ELBO :  tensor(16512.6935)
ELBO :  tensor(16512.9282)
ELBO :  tensor(16513.1694)
ELBO :  tensor(16513.7139)
ELBO :  tensor(16514.1672)
ELBO :  tensor(16514.4951)
ELBO :  tensor(16514.5859)
ELBO :  tensor(16514.4860)
ELBO :  tensor(16514.3452)
ELBO :  tensor(16514.1997)
ELBO :  tensor(16514.1157)
ELBO :  tensor(16514.1236)
ELBO :  tensor(16514.2213)
ELBO :  tensor(16514.3483)
ELBO :  tensor(16514.4201)
ELBO :  tensor(16514.3951)
ELBO :  tensor(16514.2873)
ELBO :  tensor(16514.1519)
ELBO :  tensor(16514.0686)
ELBO :  tensor(16514.1063)
ELBO :  tensor(16514.1947)
ELBO :  tensor(16514.4020)
ELBO :  tensor(16514.5365)
ELBO :  tensor(16514.6486)
ELBO :  tensor(16514.6668)
ELBO :  tensor(16514.6242)
E

ELBO :  tensor(16514.7138)
ELBO :  tensor(16514.6657)
ELBO :  tensor(16514.5895)
ELBO :  tensor(16514.4849)
ELBO :  tensor(16514.3170)
ELBO :  tensor(16514.1159)
ELBO :  tensor(16513.8116)
ELBO :  tensor(16513.5077)
ELBO :  tensor(16513.0411)
ELBO :  tensor(16512.5282)
ELBO :  tensor(16511.6479)
ELBO :  tensor(16510.8653)
ELBO :  tensor(16510.0219)
ELBO :  tensor(16510.4315)
ELBO :  tensor(16511.5474)
ELBO :  tensor(16513.3915)
ELBO :  tensor(16514.5523)
ELBO :  tensor(16514.3847)
ELBO :  tensor(16513.5078)
ELBO :  tensor(16512.4926)
ELBO :  tensor(16512.7829)
ELBO :  tensor(16513.4495)
ELBO :  tensor(16514.4926)
ELBO :  tensor(16514.6829)
ELBO :  tensor(16514.2353)
ELBO :  tensor(16513.7902)
ELBO :  tensor(16513.7588)
ELBO :  tensor(16514.2792)
ELBO :  tensor(16514.6206)
ELBO :  tensor(16514.6664)
ELBO :  tensor(16514.3976)
ELBO :  tensor(16514.2153)
ELBO :  tensor(16514.3546)
ELBO :  tensor(16514.6008)
ELBO :  tensor(16514.7813)
ELBO :  tensor(16514.6906)
ELBO :  tensor(16514.5228)
E

In [48]:
def vec2mat_lower_tridiag(vectC): 
    '''
    transform a vector of size (n(n+1)/2) into a lower tridiagonal matrix of size (n,n). 
    
    args : 'vectC' vector of size (n*(n+1)/2)
    
    returns 'matC' : matrice of size(n,n). Its lower tridiagonal will be computed from 
            vectC as follows : 
            for all i < n : - matC[i,i] = out[i*(i+1)/2]
                            - matC[i+1, 0] = out[i*(i+1)/2 + 1]
            matC[n,n] = out[n*(n+1)/2] 
            
            see torch.tril_indices for more 
            
    Note that if x is an array of size (n*(n+1)/2) for any integer n, 
    then mat2vec_lower_tridiag(vec2mac_lower_trig(x)) = x 
    '''
    
    somme_entiers = vectC.shape[0]
    n = int((-1+math.sqrt(1+8*somme_entiers))/2)
    mask = torch.tril_indices(n,n).unbind()
    matC = torch.zeros((n,n))#, dtype = torch.float)
    matC[mask] = vectC
    return matC

def mat2vec_lower_tridiag(matC): 
    '''
    extract the lower tridiagonal of a matrice. returns a vector. 
    
    args : 'matC' : A squared matrices of float. 
    
    returns : a vector of size(n*(n+1)/2) where n is the first (or second) size of matC. 
              It is computed as the following : 
              for all i < n : -out[i*(i+1)/2] = matC[i,i]
                              -out[i*(i+1)/2 + 1] = matC[i+1, 0]
              out[n*(n+1)/2] = matC[n,n]
              
    Note that if x is an array of size (n*(n+1)/2) for any integer n, 
    then mat2vec_lower_tridiag(vec2mac_lower_trig(x)) = x 
    
    see torch.tril_indices for more 
    '''
    n = matC.shape[0]
    mask = torch.tril_indices(n,n).unbind()
    return matC[mask]

In [33]:
print(model.optimizer.param_groups[0]['params'][1].shape)
model.params['C'].requires_grad = False

torch.Size([4, 30])


In [52]:
d = {'a': 27, 'b': 54, 'c': 5}
print(d.items())

TypeError: 'dict_items' object is not subscriptable

In [166]:
def torch_gradient_ascent(optimizer, compute_gain,params,  lr , tolerance , N_epoch , requires_closed_beta, requires_closed_Sigma, verbose): 
    '''
    gradient ascent function. We compute the gradients thanks to the autodifferentiation of pytorch. 

    args : 
            'optimizer' : torch.optim.optimizer. the optimizer for the parameters. 
            
            'compute_gain' : function. It should call the parameters by itself. i.e. we wil call compute_gain() without any parameter 
                            in argument. 

            'lr' : float.  a learning rateM if we want to set the optimizer learning rate to a certain lr. If None, 
                  it will take the actual learning_rate of the optimizer. 
            'tolerance': float. the threshold we set to stop the algorithm. It will stop if the norm of each gradient's parameter 
                         is lower than this threshold, or if we are not improving the loss more than tolerance. 
            'N_epoch': int. the Maximum number of epoch we are ready to do. 

            'Verbose' : bool. if True, will print some messages useful to interpret the gradient ascent. If False, nothing will be printed. 
            
            we have ot yet implement it so thtat it can takes batches. 


    returns : the parameters optimized. 
    '''

    t0 = time.time()
    
    MSE_Sigma_list = list()
    MSE_beta_list = list()
    ELBO_list = list()
    running_times = list()
    # we set the gradient to zero just to make sure the gradients are properly calculated
    optimizer.zero_grad()
    if lr is not None : # if we want to set a threshold, we set it. Ohterwise, we skip this condition and keep the actual learning_rate
        optimizer.param_groups[0]['lr'] = lr 

        
    #if requires_closed_beta : 
        
    '''
    #if batch_size is None, we take n. 
    if batch_size == None : 
        batch_size = self.Y.shape[0]
    '''

    stop_condition = False 
    i = 0
    old_loss = 1.

    while i < N_epoch and stop_condition == False: 
        optimizer.zero_grad()
        loss = -compute_gain()
        loss.backward()
        '''
        with torch.no_grad(): 
            model.C = mat2vec_lower_tridiag(torch.cholesky(model.Sigma))
            print('ELBO : ', model.compute_ELBO_tridiag())
        '''
        #print('sanity check : ', torch.norm(params['C'].grad+grad_C(data[0], data[1],data[2] ,params['M'] ,params['S'] ,params['C'] ,params['beta']) ))
        if torch.isnan(loss).item() == True : 
            print('NAN')
        else : last_params = params 
        optimizer.step()
        i += 1
        
        '''
        tmp = closed_Sigma(params['M'],params['S'])
        print('model.C before : ', model.C)
        params['C'] = mat2vec_lower_tridiag(torch.cholesky(tmp))
        print('model.C after : ', model.C)
        '''
        # condition to see if we have reach the tolerance threshold
        if  abs(loss.item() - old_loss) < tolerance : 
            #if max([torch.norm(param.grad) for param in params]) < tolerance  or abs(loss.item()- old_loss)>  tolerance :
            stop_condition = True 
        old_loss = loss.item()
        
        running_times.append(time.time()-t0)
        
        if requires_closed_Sigma : 
            print('formula with Sigma')
            if i % 100 == 0 : 
                params['Sigma'] = closed_Sigma(params['M'],params['S'])
            MSE_Sigma_list.append(torch.mean((params['Sigma']-true_Sigma)**2).item())
        
        else : 
            
            if len(params['C'].shape) > 1 : 
                MSE_Sigma_list.append(torch.mean((torch.mm(params['C'],params['C'].T)-true_Sigma)**2).item())
            else : 
                MSE_Sigma_list.append(torch.mean((torch.mm(vec2mat_lower_tridiag(params['C']),
                                                       vec2mat_lower_tridiag(params['C']).T)-true_Sigma)**2).item())
        
        
        #print('grad_Sigma: ', grad_Sigma(data[0],data[1],data[2],params['M'],params['S'],params['Sigma'], params['beta']))
        MSE_beta_list.append(torch.mean((params['beta']-true_beta)**2).item())
        ELBO_list.append(-loss.item())
        
        if i%100 == 0 and verbose : 
            print('iteration number: ', i)
            print('-------UPDATE-------')
            print(' MSE with Sigma : ', np.round(MSE_Sigma_list[-1],5))
            print(' MSE with beta : ', np.round(MSE_beta_list[-1],5))
            print('ELBO : ', np.round(-loss.item(),5))
            print_stats(loss, last_params, optimizer)
            
    
    # just print some stats if we want to 
    if stop_condition : 
        print('---------------------------------Tolerance {} reached in {} iterations'.format(tolerance, i))
    else : 
        print('---------------------------------Maximum number of iterations reached : ', N_epoch)
        print_stats(loss, last_params, optimizer)
    print(' MSE with Sigma : ', np.round(MSE_Sigma_list[-1],5))
    print(' MSE with beta : ', np.round(MSE_beta_list[-1],5))
    
    return last_params, running_times, MSE_Sigma_list, MSE_beta_list, ELBO_list

def print_stats(loss, params, optimizer): 
    '''
    small function that print some stats. 

    It will print the actual learning rate of the optimizer, the actual log likelihood 
    and the norms of each parameter's gradient. The norm of the parameter's gradient should be low
    when we are close to the optimum. 
    '''
    print('---------------------------------lr :', optimizer.param_groups[0]['lr'])
    print('---------------------------------log likelihood :', - loss.item())
    for param_name, param in params.items(): 
        try :
            with torch.no_grad : # to avoid Warnings 
                print('---------------------------------grad_{}_norm : '.format(param_name), round(torch.norm(param.grad).item(), 3))
        except : 
            pass 

def ELBO(Y, O,covariates ,M ,S ,C ,beta): 
    n = Y.shape[0]
    SrondS = torch.multiply(S,S)
    OplusM = O+M
    MmoinsXB = M-torch.mm(covariates, beta) 
    
    tmp = torch.sum(  torch.multiply(Y, OplusM)  -torch.exp(OplusM+SrondS/2) +1/2*torch.log(SrondS))
    
    tmp -= 1/2*torch.trace(  
                            torch.mm(  
                                        torch.inverse(torch.mm(C,C.T)), 
                                        torch.diag(torch.sum(SrondS, dim = 0))+ torch.mm(MmoinsXB.T, MmoinsXB)
                                    )
                          )
    tmp-= n*torch.log(torch.det(C))
    return tmp 

def ELBO_tridiag(Y, O,covariates ,M ,S ,vectC ,beta):
    matC = vec2mat_lower_tridiag(vectC)
    return ELBO(Y, O,covariates ,M ,S ,matC ,beta)

def grad_beta(Y, O, covariates ,M ,S ,C ,beta) : 
    grad = torch.mm(torch.mm(covariates.T, M-torch.mm(covariates, beta) ), torch.inverse(torch.mm(C,C.T)))
    return grad 

def grad_M(Y, O, covariates ,M ,S ,C ,beta):
    grad = Y - torch.exp(O+M+torch.multiply(S,S)/2)-torch.mm(M-torch.mm(covariates,beta), torch.inverse(torch.mm(C,C.T)))
    return grad 
def grad_S(Y, O, covariates ,M ,S ,C ,beta): 
    return torch.div(1,S)-torch.multiply(S, torch.exp(O+M+torch.multiply(S,S)/2))-torch.mm(S, torch.diag(torch.diag(torch.inverse(torch.mm(C,C.T)))))

def grad_C(Y, O, covariates ,M ,S ,C ,beta): 
    n = Y.shape[0]
    CCT = torch.mm(C,C.T)
    MmoinsXB = M-torch.mm(covariates, beta) 
    big_mat = torch.diag(torch.sum(torch.multiply(S,S), dim = 0))+ torch.mm(MmoinsXB.T, MmoinsXB)
    return torch.mm(torch.mm(torch.inverse(C),(big_mat +big_mat.T)/2.),torch.inverse(CCT)).T - Y.shape[0]*torch.inverse(C).T

def grad_Sigma(Y, O, covariates ,M ,S ,Sigma ,beta): 
    n = Y.shape[0]
    inv_Sigma = torch.inverse(Sigma)
    grad = -n/2*(inv_Sigma)
    grad += 1/2*(sum([inv_Sigma@(torch.outer(M[i,:],M[i,:])+ torch.diag(torch.multiply(S,S)[i,:]))@inv_Sigma 
                      for i in range(n)]))
    return grad


def closed_Sigma(M,S):
    n = M.shape[0]
    return 1/n*(torch.sum(torch.stack([torch.outer(M[i,:],M[i,:]) + torch.diag(torch.multiply(S,S)[i,:]) 
                                       for i in range(n)]), axis = 0))
                          
def grad_test(Y_, O_, covariates_,M_ ,S_ ,C_ ,beta_): 
    Y = torch.clone(Y_)
    O = torch.clone(O_)
    covariates = torch.clone(covariates_)
    M = torch.clone(M_)
    S = torch.clone(S_)
    C = torch.clone(C_)
    beta = torch.clone(beta_)
    for i in range(500): 
        grad = grad_C(Y, O, covariates ,M ,S ,C ,beta)
        C+=0.00000015*grad 
        #M+=0.00015*grad 
        #beta+=0.00000015*grad
        if torch.isnan(ELBO(Y, O, covariates ,M ,S ,C ,beta)) == True : 
            print('nan')
        if i % 200 == 0: 
            print('ELBO : ', ELBO(Y, O, covariates ,M ,S ,C ,beta))
        #print('norm grad : ', torch.norm(grad))
def ELBO_Sigma(Y, O,covariates ,M ,S ,Sigma ,beta):
    n = Y.shape[0]
    SrondS = torch.multiply(S,S)
    OplusM = O+M
    MmoinsXB = M-torch.mm(covariates, beta) 
    tmp = torch.sum(  torch.multiply(Y, OplusM)  -torch.exp(OplusM+SrondS/2) +1/2*torch.log(SrondS))
    tmp -= 1/2*torch.trace(  
                            torch.mm(  
                                        torch.inverse(Sigma_init), 
                                        torch.diag(torch.sum(SrondS, dim = 0))+ torch.mm(MmoinsXB.T, MmoinsXB)
                                    )
                          )
    tmp-= n/2*torch.log(torch.det(Sigma))
    return tmp
#grad_test(Y_sampled, O, covariates, M_init,S_init, C_init, beta_init )
#print(grad_Sigma(Y_sampled, O, covariates, M_init,S_init, closed_Sigma(M_init,S_init), beta_init))



$$\boxed{\begin{align} J_{\theta, q}(Y) &=\mathbb{1}_n^{\top}\left(Y \odot(O+M)-\frac{1}{2} \exp \left(O+M+\frac{S\odot S}{2}\right) + \frac 12 \log (S \odot S) \right)\mathbb{1}_p \\
& \quad  - \frac 12\operatorname{tr}\left((CC^{\top})^{-1}\left(\operatorname{diag}(\mathbb{1}_n^{\top} (S\odot S))+(M-X \beta)^{\top}(M-X \beta)\right)\right) \\
& \quad  - n \log |C|+ cst 
\end{align}}$$

$$\boxed{\begin{align} J_{\theta, q}(Y) &=\mathbb{1}_n^{\top}\left(Y \odot(O+M)-\frac{1}{2} \exp \left(O+M+\frac{S\odot S}{2}\right) + \frac 12 \log (S \odot S) \right)\mathbb{1}_p \\
& \quad  - \frac 12\operatorname{tr}\left(\Sigma^{-1}\left(\operatorname{diag}(\mathbb{1}_n^{\top} (S\odot S))+(M-X \beta)^{\top}(M-X \beta)\right)\right) \\
& \quad  - \frac n2 \log |\Sigma|+ cst 
\end{align}}$$

In [173]:
model = PLN_full(C_init, beta_init, M_init, S_init)
%time model.full_grad_ascent(data, N_epoch = 6000,verbose=True, lr = 0.1)

iteration number:  100
-------UPDATE-------
 MSE with Sigma :  5.61132
 MSE with beta :  0.33093
ELBO :  163239.08193
---------------------------------lr : 0.1
---------------------------------log likelihood : 163239.08193271194
iteration number:  200
-------UPDATE-------
 MSE with Sigma :  2.45368
 MSE with beta :  0.33908
ELBO :  165014.5375
---------------------------------lr : 0.1
---------------------------------log likelihood : 165014.53749652667
iteration number:  300
-------UPDATE-------
 MSE with Sigma :  1.31774
 MSE with beta :  0.3024
ELBO :  165265.02398
---------------------------------lr : 0.1
---------------------------------log likelihood : 165265.02398174725
iteration number:  400
-------UPDATE-------
 MSE with Sigma :  0.81393
 MSE with beta :  0.26127
ELBO :  165380.808
---------------------------------lr : 0.1
---------------------------------log likelihood : 165380.80799704787
iteration number:  500
-------UPDATE-------
 MSE with Sigma :  0.56117
 MSE with beta : 

iteration number:  3700
-------UPDATE-------
 MSE with Sigma :  0.08814
 MSE with beta :  0.07253
ELBO :  165739.7185
---------------------------------lr : 0.1
---------------------------------log likelihood : 165739.7185038493
iteration number:  3800
-------UPDATE-------
 MSE with Sigma :  0.08804
 MSE with beta :  0.07241
ELBO :  165739.58144
---------------------------------lr : 0.1
---------------------------------log likelihood : 165739.5814415583
iteration number:  3900
-------UPDATE-------
 MSE with Sigma :  0.08809
 MSE with beta :  0.07238
ELBO :  165739.80436
---------------------------------lr : 0.1
---------------------------------log likelihood : 165739.8043636381
iteration number:  4000
-------UPDATE-------
 MSE with Sigma :  0.0881
 MSE with beta :  0.07239
ELBO :  165740.29163
---------------------------------lr : 0.1
---------------------------------log likelihood : 165740.29163357735
iteration number:  4100
-------UPDATE-------
 MSE with Sigma :  0.08793
 MSE with bet

In [170]:
d = 4 # nb of cavariates
n = 400; p = 20

In [171]:
true_Sigma = torch.from_numpy(toeplitz(0.5**np.arange(p)))
true_C = torch.cholesky(true_Sigma)
true_beta = torch.randn(d, p)

covariates = torch.rand((n,d))
O =  1+torch.zeros((n,p))

sample_model = sample_PLN()
Y_sampled = torch.from_numpy(sample_model.sample(true_Sigma,true_beta, O, covariates)) 

data = [Y_sampled.double(), O, covariates]

In [172]:
torch.manual_seed(0)

noise = torch.randn(p) 
Sigma_init =  torch.diag(noise**2)
#Sigma_init = torch.from_numpy(toeplitz(0.4**np.arange(p)))
C_init = torch.cholesky(Sigma_init)
beta_init = torch.rand((d, p))

M_init = torch.ones((n,p))/100# some random values to initialize we divide to avoid nan values 
S_init = torch.ones((n,p))/8 # some random values to initializ. we divise to avoid nan values 

Here we have a PLN model described as the following : 

- Consider $n$ sites $(i=1 \ldots n)$

- Measure $x_{i}=\left(x_{i h}\right)_{1 \leq h \leq d}$ :
$x_{i h}=$ given environmental descriptor (covariate) for site $i$
(altitude, temperature, latitude, ...)

- Consider $p$ species $(j=1 \ldots p)$ Measure $Y=\left(Y_{i j}\right)_{1 \leq i \leq n, 1 \leq j \leq p}$ :

- Measure $Y = Y_{i j}=$ number of observed individuals from species $j$ in site $i$ (abundance). 

- Associate a random vector $Z_{i}$ with each site Assume that the unknown $\left(Z_{i}\right)_{1 \leq i \leq n}$ are independant such that:
$$
Z_{i} \sim \mathcal{N}_{p}(x_i \beta, \Sigma) \quad \Sigma = CC^{\top}
$$

and $C$ is a lower triangular matrix. 
- Assume that the observed abundances $\left(Y_{i j}\right)_{1 \leq i \leq n, 1 \leq j \leq p}$ are independent conditionally on the $Z=\left(Z_{i}\right)_{i}$


$$
\left(Y_{i j} \mid Z_{i j}\right) \sim \mathcal{P}\left(\exp \left(o_{i j}+Z_{i j}\right)\right)
$$

Where $O = (o_{ij})_{1\leq i\leq n, 1\leq j\leq p}$ are known offsets. 

The unknown parameter is $\theta = (C,\beta)$.

$Z$ being a latent variable, we want to use the EM algorithm to derive the maximum likelihood estimator. However, it requires to compute 

$$
\mathbb{E}_{\theta}\left[p_{\theta}\left(Z_{i} \mid Y\right)\right]=\mathbb{E}_{\theta}\left[p_{\theta}\left(Z_{i} \mid Y_{i}\right)\right] \propto \int_{\mathbb{R}^{p}} p_{\theta}\left(Z_{i}\right) \prod_{j} p_{\theta}\left(Y_{i j} \mid Z_{i j}\right) \mathrm{d} Z_{i}
$$ which is intractable in practice. 

We thus choose the variationnal approximation. We set

$$ 
q^{\star} = \underset{q \in \mathcal{Q_{gauss}}}{\operatorname{argmax}} J_{\theta,q}(Y) 
$$
<br>
$$
\begin{align} J_{\theta, q}(Y)& =\log p_{\theta}(Y)-K L\left[q(Z) \| p_{\theta}(Z \mid Y)\right]                                    \\ 
                              & = \mathbb{E}_{q}\left[\log p_{\theta}(Y, Z)\right] \underbrace{-\mathbb{E}_{q}[\log q(Z)]}_{\text {entropy } \mathcal{H}(q)}    \end{align}
$$

where 

$$
\mathcal{Q}_{\text {Gauss }}=\{
q=\left(q_{1}, \ldots q_{n}\right), q_{i} \sim \mathcal{N}\left(M_{i}, \operatorname{diag} (S_{i}\odot S_i ), M_i \in \mathbb{S} ^p, S_i \in \mathbb{R} ^p\right)\}
$$


The Variational EM (VEM) consists in alternate between two steps : 
- VE step: update $q$
$$
q^{h+1}=\underset{q \in \mathcal{Q_{gauss}}}{\arg \max } J_{\theta^{h}, q}(Y)=\underset{q \in \mathcal{Q_{gauss}}}{\arg \min } K L\left[q(Z) \| p_{\theta^{h}}(Z \mid Y)\right]
$$
- M step: update $\theta$
$$
\theta^{h+1}=\underset{\theta}{\arg \max } J_{\theta, q^{h+1}}(Y)=\underset{\theta}{\arg \max } \mathbb{E}_{q^{h+1}}\left[\log p_{\theta}(Y, Z)\right]
$$

Let's compute the ELBO $J_{\theta, q}(Y)$


$$
J_{\sigma, q}(Y)=\underbrace{\mathbb{E}_{q}\left[\log p_{\theta}(Y \mid Z)\right]}_{(1)}+\underbrace{E_{q}\left[\log p_{\theta}(Z)\right]}_{(2)}+\underbrace{H(q)}_{(3)}
$$
$$
\begin{align}
(1)& =\sum_{i} \mathbb{E}_{q}\left[\log p_{\theta}\left(Y_{i} \mid Z\right)\right] \\
&=\sum_{i} \mathbb{E}_{q}\left[\log p_{\theta}\left(Y_{i} \mid Z_{i}\right)\right] \\
&=\sum_{i, j} \mathbb{E}_{q}\left[\log p_{\theta}\left(Y_{i j} \mid Z_{i j}\right)\right] \\
& =\sum_{i, j} \mathbb{E}_{q}\left[Y_{i j}\left(o_{i j}+Z_{i j}\right)-\exp \left(o_{i j}+Z_{i j}\right)\right] + cst
\end{align}
$$

We need to compute some moments of $Z$ under $q$. 

$$
\mathbb{E}_{q}\left[Z_{i j}\right]=M_{i j} \quad \quad E_{q}\left[\operatorname{exp}\left(Z_{i j}\right)\right]=\frac{1}{2} \operatorname{exp}\left(M_{i j}+\frac{(S_{ij})^2}{2}\right)
$$

So that 

$$
\begin{aligned}
(1) &=\sum_{i, j} Y_{i j}\left(o_{i j}+M_{i j}\right)-\frac{1}{2} \exp \left(o_{i j}+M_{i j}+\frac{(S_{i_{jj}})^2}{2}\right) + cst \\
&=\mathbb{1}_n^{\top}\left(Y \odot(O+M)-\frac{1}{2} \exp \left(O+M+\frac{S \odot S}{2}\right)\right)\mathbb{1}_p + cst 
\end{aligned}
$$



Where we have denoted $M = (M_{ij})_{1\leq i\leq n, 1\leq j\leq p}$ and $ S = (S_{ij})_{1\leq i\leq n, 1\leq j\leq p}$. The exponential is applied pointwise on the last equation. 

$$
\begin{aligned}
(3)=H(q) &=\sum_{i} H\left(q_{i}\right) \\
&=\sum_{i} \log \left(\sqrt{(2 \pi e)^{p}\left|S_{i}\right|}\right) \\
&=\frac 12\sum_{i} \log \left|(S_{i} \odot S_i)^2 \right|+cst \\
&=\sum_{i j} \log S_{i j}+cst \\
& =  \mathbb{1}_n ^{\top}(\log S )\mathbb{1}_p+cst\\
\end{aligned}
$$

Where the log is applied pointwise at the last equation. 

$$
\begin{aligned}
(2)=\mathbb{E}_{q}\left[\log p_{\theta}(Z)\right] &=\sum_{1} E_{q}\left[\log p_{\theta}\left(Z_{i}\right)\right] \\
&=-\frac{n}{2} \log |\Sigma|+\sum_{i} \mathbb{E}_{q}\left[-\frac{1}{2}\left(Z_{i}-X_{i} \beta\right)^{\top} \Sigma^{-1}\left(Z_{i} - X_{i} \beta\right)\right] +cst
\end{aligned}
$$

$
\text { Let } V \sim \mathcal{N} \left(\mu, \Lambda), \mu \in \mathbb{R}^{p}, \Lambda \in \mathcal S _p ^{++}\right.  \\
\text { Let's compute } \; \mathbb{E}\left[V^{\top} \Sigma^{-1} V\right]
$

We denote $\Sigma ^{-1 / 2}$ the square root Matrix of $\Sigma^{-1}$. It exists since $\Sigma ^{-1} \in \mathcal{S}_p^{++}$. 

$$
\begin{aligned}
\mathbb{E}\left[V^{\top} \Sigma^{-1} V\right] &=\mathbb{E}\left[V^{\top}\Sigma ^{-1 / 2} \Sigma^{-1 / 2} V\right]\\
&=\mathbb{E}\left[\left(\Sigma^{-1 / 2} V\right)^{\top}\left(\Sigma^{-1 / 2} V\right)\right] \\
&=\mathbb{E} \|\Sigma^{-1 / 2} V \|_{2}^{2}
\end{aligned}
$$

$
\text {Let } \tilde{V}=\Sigma^{-1 / 2} V, \quad \tilde{V} \sim \mathcal{N}\left(\Sigma^{-1 / 2} \mu,  \Sigma^{-1 / 2} \Lambda \Sigma^{-\frac{1}{2}}\right)
$

$$
\begin{aligned}
\mathbb{E}\left[V^{\top} \Sigma^{-1} V\right] &=\mathbb{E}\|\widetilde{V}\|_{2}^{2} \\
&=\sum \mathbb E \widetilde{V}_{j}^{2} \\
&=\sum \operatorname{var}\left(\widetilde{V}_{j}\right)^{2}+\mathbb{E}\left[\widetilde{V}_{j}\right]^{2}\\
&=\sum_{j}\left(\Sigma^{-1 / 2} \Lambda \Sigma^{-1 / 2}\right)_{j j}+\left(\Sigma^{-1 / 2} \mu\right)^{2}_j\\
&= \operatorname{tr}\left(\Sigma^{-1 / 2} \Lambda \Sigma^{-1 / 2}\right)+\sum_{j}\left(\left(\Sigma_{j,.}^{-1 / 2}\right)^{\top} \mu\right)^{2}\\
&=\operatorname{tr}\left(\Sigma^{-1} \Lambda\right)+\sum_{j}\left(\left(\Sigma_{j,.}^{-1/2}\right)^{\top} \mu\right)^2
\end{aligned}
$$

Since under $q$, $Z_{i}-X_{i} \beta \sim \mathcal N (M_i - X_i \beta, S_i \odot S_i ) $
$$
\begin{aligned}
(2) &=- \frac 12 \sum_{i} \operatorname{tr}\left(\Sigma^{-1} (S_{i} \odot S_i) \right) - \frac 12 \sum_{i, j}\left(\left(\Sigma_{j,.}^{-1 / 2} \right) ^{\top}\left(M_{i}-X_{i} \beta\right)\right)^2  - \frac n2 \log |\Sigma|+ cst  \\
&= - \frac 12 \operatorname{tr}\left(\Sigma^{-1}\left(\sum_{i} S_{i}\odot S_i \right)\right) - \frac 12 \sum_{i, j}\left(\Sigma^{-1 / 2}(M-X \beta)\right)_{j, i}^{\top}\left(\Sigma
^{-1 / 2}(M-X \beta)\right)_{i, j}  - \frac n2 \log |\Sigma|+ cst  \\
&=- \frac 12 \operatorname{tr}\left(\Sigma^{-1}\left(\sum_{i} S_{i} \odot S_i  \right)\right)- \frac 12 \operatorname{tr}\left(\Sigma^{-1 / 2}(M-X \beta)^{\top}(M-X \beta) \Sigma^{-1 / 2}\right)  - \frac n2 \log |\Sigma|+ cst  \\
&=- \frac 12 \operatorname{tr}\left(\Sigma^{-1}\left(\operatorname{diag}(\mathbb{1}_n^{\top} (S\odot S))+(M-X \beta)^{\top}(M-X \beta)\right)\right)  - \frac n2 \log |\Sigma|+ cst 
\end{aligned}
$$
We then have : 


$$\boxed{\begin{align} J_{\theta, q}(Y) &=\mathbb{1}_n^{\top}\left(Y \odot(O+M)-\frac{1}{2} \exp \left(O+M+\frac{S\odot S}{2}\right) + \frac 12 \log (S \odot S) \right)\mathbb{1}_p \\
& \quad  - \frac 12\operatorname{tr}\left(\Sigma^{-1}\left(\operatorname{diag}(\mathbb{1}_n^{\top} (S\odot S))+(M-X \beta)^{\top}(M-X \beta)\right)\right) \\
& \quad  - \frac n2 \log |\Sigma|+ cst 
\end{align}}$$